# Introdução ao Django: Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_django_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

# Validações

**1. Classe `Validador` e subclasses:**

- A classe `Validador` e suas subclasses (`ValidadorNome`, `ValidadorIdade`, `ValidadorSexo`, `ValidadorEstadoCivil`, `ValidadorTipoSanguineo`) implementam a lógica de validação para os campos `nome`, `idade`, `sexo`, `estado_civil` e `tipo_sanguineo`, respectivamente.
    
- Cada validador verifica se o valor do campo atende aos critérios especificados e levanta uma exceção `ValidationError` se a validação falhar.

- O método `validar` em cada classe da cadeia chama o próximo validador na cadeia, se houver um. Isso permite que vários validadores sejam encadeados para realizar múltiplas validações em um único campo.


**Arquivo `validador.py`:**

In [ ]:
# arquivo: validador.py
from django.core.exceptions import ValidationError
import re

class Validador:
    """
    Classe base para os validadores.
    """
    def __init__(self, proximo_validador=None):
        self.proximo_validador = proximo_validador

    def validar(self, value, field_name):
        """
        Método para validar o valor de um campo.
        Deve ser implementado nas subclasses.

        Args:
            value: O valor a ser validado.
            field_name (str): O nome do campo que está sendo validado.

        Returns:
            None: Se a validação for bem-sucedida.

        Raises:
            ValidationError: Se a validação falhar.
        """
        if self.proximo_validador:
            self.proximo_validador.validar(value, field_name)

class ValidadorNome(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, str) or not value.strip():
            raise ValidationError(f"{field_name} deve ser uma string não vazia.")

class ValidadorIdade(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, int):
            raise ValidationError(f"{field_name} deve ser um número inteiro.")
        if value <= 0 or value > 125:
            raise ValidationError(f"{field_name} deve estar entre 0 e 125.")

class ValidadorSexo(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, str) or value not in ["M", "F"]:
            raise ValidationError(f"{field_name} deve ser 'M' ou 'F'.")

def validar_cpf(value):
    """Função para validar o CPF."""
    # Remove caracteres não numéricos
    cpf = ''.join(filter(str.isdigit, value))
    if len(cpf) != 11:
        raise ValidationError("CPF deve ter 11 dígitos.")
    # Lógica de validação do CPF (adaptada de https://www.pythonbrasil.org.br/wiki/VerificadorDeCPF)
    try:
        int(cpf)  # Verifica se todos os caracteres são números
    except ValueError:
        raise ValidationError("CPF deve conter apenas números.")

    # Cálculo do primeiro dígito verificador
    soma = 0
    for i in range(9):
        soma += int(cpf[i]) * (10 - i)
    resto = soma % 11
    digito1 = 0 if resto < 2 else 11 - resto

    if digito1 != int(cpf[9]):
        raise ValidationError("CPF inválido.")

    # Cálculo do segundo dígito verificador
    soma = 0
    for i in range(10):
        soma += int(cpf[i]) * (11 - i)
    resto = soma % 11
    digito2 = 0 if resto < 2 else 11 - resto

    if digito2 != int(cpf[10]):
        raise ValidationError("CPF inválido.")
    return value

class ValidadorEstadoCivil(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, str):
            raise ValidationError(f"{field_name} deve ser uma string.")

class ValidadorTipoSanguineo(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, str) or value not in ["A+", "A-", "B+", "B-", "AB+", "AB-", "O+", "O-"]:
            raise ValidationError(f"{field_name} inválido.")

**Arquivo `models.py`:**

In [ ]:
# arquivo: models.py
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator, ValidationError
from datetime import date
import re  # Importe o módulo re para validar o CPF
from .validador import ValidadorNome, ValidadorIdade, ValidadorSexo, validar_cpf, ValidadorEstadoCivil, ValidadorTipoSanguineo # Importe os validadores do arquivo validators.py


class Pessoa(models.Model):
    id = models.AutoField(primary_key=True)
    nome = models.CharField(max_length=255)
    idade = models.IntegerField(
        validators=[
            MinValueValidator(0, message="A idade deve ser maior ou igual a 0."),
            MaxValueValidator(125, message="A idade deve ser menor ou igual a 125."),
        ]
    )
    sexo = models.CharField(max_length=1, choices=[('M', 'Masculino'), ('F', 'Feminino')])
    data_nascimento = models.DateField()
    cidade_natal = models.CharField(max_length=100)
    estado_natal = models.CharField(max_length=50)
    cpf = models.CharField(max_length=14, unique=True, validators=[validar_cpf])
    profissao = models.CharField(max_length=100, blank=True, null=True)
    cidade_residencia = models.CharField(max_length=100)
    estado_residencia = models.CharField(max_length=50)
    estado_civil = models.CharField(max_length=50, blank=True, null=True)

    def clean(self):
        super().clean()
        validador_nome = ValidadorNome()
        validador_idade = ValidadorIdade(validador_nome)
        validador_sexo = ValidadorSexo(validador_idade)

        validador_nome.validar(self.nome, 'Nome')
        validador_idade.validar(self.idade, 'Idade')
        validador_sexo.validar(self.sexo, 'Sexo')

    class Meta:
        abstract = True  # Classe base abstrata, não será criada tabela no banco

    def __str__(self):
        return f"{self.nome} ({self.cpf})"

    @classmethod
    def cadastrar(cls, **kwargs):
        """
        Método de classe para cadastrar uma pessoa.
        Deve ser implementado nas subclasses.
        """
        raise NotImplementedError("O método cadastrar deve ser implementado na subclasse.")



class Doador(Pessoa):
    contato_emergencia = models.CharField(max_length=255)
    tipo_sanguineo = models.CharField(max_length=5, choices=[
        ('A+', 'A+'), ('A-', 'A-'), ('B+', 'B+'), ('B-', 'B-'),
        ('AB+', 'AB+'), ('AB-', 'AB-'), ('O+', 'O-'), ('O-', 'O-')
    ])
    # cpf = models.CharField(max_length=14, unique=True, validators=[validar_cpf]) # Usa o validador customizado

    def clean(self):
        super().clean()
        validador_estado_civil = ValidadorEstadoCivil()
        validador_tipo_sanguineo = ValidadorTipoSanguineo(validador_estado_civil)
        validador_estado_civil.validar(self.estado_civil, 'Estado Civil')
        validador_tipo_sanguineo.validar(self.tipo_sanguineo, 'Tipo Sanguíneo')

    @classmethod
    def cadastrar(cls, nome, idade, sexo, data_nascimento, cidade_natal, estado_natal, cpf, profissao, cidade_residencia, estado_residencia, estado_civil, contato_emergencia, tipo_sanguineo):
        """
        Método de classe para cadastrar um doador.
        """
        doador = cls(
            nome=nome,
            idade=idade,
            sexo=sexo,
            data_nascimento=data_nascimento,
            cidade_natal=cidade_natal,
            estado_natal=estado_natal,
            cpf=cpf,
            profissao=profissao,
            cidade_residencia=cidade_residencia,
            estado_residencia=estado_residencia,
            estado_civil=estado_civil,
            contato_emergencia=contato_emergencia,
            tipo_sanguineo=tipo_sanguineo,
        )
        doador.full_clean()  # Valida o model
        doador.save()  # Salva o doador no banco de dados
        return doador

**2. Validação no método `clean` do Model:**

- O método `clean` é um método do Django Model que permite realizar validações personalizadas em um Model.
    
- Nos Models `Pessoa` e `Doador`, o método `clean` foi sobrescrito para executar a cadeia de validadores.
    
- A cadeia de validadores é criada instanciando os validadores na ordem em que devem ser executados e passando o próximo validador na cadeia para o construtor do validador anterior.
    
- O método `validar` do primeiro validador na cadeia é chamado, passando o valor do campo a ser validado. Se a validação falhar em algum ponto da cadeia, uma exceção `ValidationError` será levantada.

**3. Validação do CPF:**

- A função `validar_cpf` foi mantida para realizar a validação específica do CPF.
    
- Essa função é usada como um validador no campo cpf do Model Pessoa.

**4. Chamada de `full_clean()` no método cadastrar:**    
    
- No método `cadastrar` do Model `Doador`, o método `full_clean()` é chamado antes de salvar o objeto no banco de dados. Isso garante que todas as validações definidas no Model, incluindo as validações da cadeia de validadores e a validação do CPF, sejam executadas.

O código agora implementa o padrão *Chain of Responsibility* para validar os campos `nome`, `idade`, `sexo`, `estado_civil` e `tipo_sanguineo` no Django Model `Pessoa` e `Doador`. A validação do CPF continua sendo feita pela função `validar_cpf`, que é usada como um validador no campo cpf do Model.